In [1]:
import requests
import pandas as pd
import json
import datetime

from datetime import datetime
from datetime import timedelta
from datetime import date
from time import strptime

from pathlib import Path
import shutil
import time
from dotenv import dotenv_values
from selenium import webdriver
from selenium.webdriver.firefox.webdriver import WebDriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from re import search

from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import *
from selenium.webdriver import Keys, ActionChains

import calendar
from re import search
import pandas.io.common
import locale

import os
import pickle
from requests.cookies import RequestsCookieJar

In [2]:
class openBrowser():
    
    def __init__(self, user_credencials: dict = dotenv_values("./data/.env")):
        
        self.user_credencials_login = user_credencials['LOGIN_USER']
        self.user_credencials_password = user_credencials['LOGIN_PASSWORD']
        self.__mkDirDownload()
        
        
        
   

    def __mkDirDownload(self) -> None:
        os.makedirs("download", exist_ok=True)
        for file in os.listdir('download'):
            os.remove(f"download\\{file}")
            
    def __initialize_driver(self):
        
        options = Options()
        options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
        options.set_preference("browser.download.folderList", 2)
        options.set_preference("browser.download.manager.showWhenStarting", False)
        options.set_preference("browser.download.dir", fr"{str(Path().absolute())}\\download")
        options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")
        driver = webdriver.Firefox(options=options)
        
        
        driver.maximize_window()
        return driver
    

    def loguinSendKeys(self):
        driver = self.__initialize_driver()
        


       
        driver.get("https://meumb.mercantil.com.br/propostas")
       
        try:
            WebDriverWait(driver,200 ).until(lambda x:  x.execute_script("return document.readyState"))
            WebDriverWait(driver,200 ).until(EC.element_to_be_clickable((By.XPATH, "//input[@id='mat-input-0']"))).send_keys(self.user_credencials_login)
            WebDriverWait(driver,200 ).until(EC.element_to_be_clickable((By.XPATH, "//input[@id='mat-input-1']"))).send_keys(self.user_credencials_password)
            WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@title='reCAPTCHA']")))
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.recaptcha-checkbox-border"))).click()

            input("Prenchar o captcha, Pressione 'ENTRAR', após, pressione qualquer tecla")
            
            mainWin = driver.current_window_handle
            driver.switch_to.window(mainWin)

            
            WebDriverWait(driver, 200).until(EC.visibility_of_element_located((By.XPATH, "//span[contains(text(), 'X409847')]")))
            
            return driver

        except TimeoutException as exc:
                raise exc


In [5]:
class downaload_reports():


    def __init__(self, driver: webdriver, date: date,):
        self.driver = driver
        self.date = date
        self.__download_integradas()

    def __scroll(self, driver: WebDriver):
        
        driver.execute_script("window.scrollBy(0, 3000)")
        time.sleep(1)
        driver.execute_script("window.scrollBy(3000, 0)")

    def acess_integradas(self):
        driver = self.driver

        driver.get('https://meumb.mercantil.com.br/propostas')
        
        WebDriverWait(driver, 200).until(EC.visibility_of_element_located((By.XPATH, "//span[contains(text(), 'Meus filtros')]"))).click()
        try:
            WebDriverWait(driver, 200).until(EC.visibility_of_element_located((By.XPATH, "//span[contains(text(), 'INTEGRADAS')]"))).click()
        except ElementClickInterceptedException:
            driver.refresh()
            WebDriverWait(driver, 200).until(EC.visibility_of_element_located((By.XPATH, "//span[contains(text(), 'Meus filtros')]"))).click()
            WebDriverWait(driver, 200).until(EC.visibility_of_element_located((By.XPATH, "//span[contains(text(), 'INTEGRADAS')]"))).click()

        return driver
    
    def __move(self, date_move: datetime.date):
        

        date_work = date_move
        file_search = 'download/'
        
        path_to_save = f"./Integradas/{date_work.year}/{date_work.month}"
        
        os.makedirs(path_to_save, exist_ok = True)
        
        while not os.path.exists(file_search + "relatorio.xlsx") and not os.path.exists(file_search + "relatorio(1).xlsx"):
            print(file_search + "relatorio.xlsx")
            time.sleep(1)
            print("Aguardando arquivo")

        if os.path.exists(file_search + "relatorio(1).xlsx"):
            while os.path.getsize(file_search + "relatorio(1).xlsx") == 0:
                continue
            shutil.move(file_search + f"relatorio(1).xlsx", f"{path_to_save}/relatorio_{'{:02d}'.format(date_work.day)}_{date_work.month}_{date_work.year}.xlsx")
        else:
            while os.path.getsize(file_search + "relatorio.xlsx") == 0:
                continue
            shutil.move(file_search + f"relatorio.xlsx", f"{path_to_save}/relatorio_{'{:02d}'.format(date_work.day)}_{date_work.month}_{date_work.year}.xlsx")

        if os.listdir(file_search) != 0:
            for path in os.listdir(file_search):
                os.remove(file_search + path)
    
    def __calendar_manipulate(self, date_work: date, driver: WebDriver):
        locale.setlocale(locale.LC_ALL, 'pt_pt.UTF-8')
        WebDriverWait(driver,200 ).until(lambda x:  x.execute_script("return document.readyState"))
        try:
            WebDriverWait(driver, 200).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".ngx-pagination")))
        except ElementClickInterceptedException:
            driver.refresh()
            WebDriverWait(driver,200 ).until(lambda x:  x.execute_script("return document.readyState"))
            WebDriverWait(driver, 200).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".ngx-pagination")))

        calendar = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mat-date-range-input-0')))
        driver.execute_script("arguments[0].click();", calendar)
        date_1 = date_work
        date_2 = date_1 + timedelta(days = 1)
        if date_1.strftime("%b %Y").upper() == driver.find_element(By.CSS_SELECTOR, '.mat-calendar-period-button > span:nth-child(1)').text :
        
            result = driver.find_elements(By.XPATH, f'//div[@class="mat-calendar-body-cell-content mat-focus-indicator"]')
            
            for i in result:
                if i.text == '{:d}'.format(date_1.day):
                    driver.execute_script("arguments[0].click();", i)
                    break
        else:
            calendario = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.mat-calendar-period-button > span:nth-child(1)')))
            driver.execute_script("arguments[0].click();", calendario)
            result = driver.find_elements(By.CSS_SELECTOR, ".mat-calendar-body > tr")
            
            for i in result:
                datas = i.find_elements(By.CSS_SELECTOR, 'td')
                datas = i.find_elements(By.TAG_NAME, 'td')
                click = False
                for j in datas:
                    time.sleep(.50)
                    if j.text == str(date_1.strftime("%Y")):
                        driver.execute_script("arguments[0].click();", j)
                        click = True
                        break
                if click:
                    break
                
                        
            resulta_month = driver.find_elements(By.CLASS_NAME, "mat-calendar-body")
            for month in resulta_month:
                result_month = month.find_elements(By.TAG_NAME, 'td')
                click = False
                for i in result_month:
                    if search(date_1.strftime("%b").upper(), i.text):
                        driver.execute_script("arguments[0].click();", i)
                        click = True
                        break
                if click:
                    break
            # iteração nos dias
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, f'//div[@class="mat-calendar-body-cell-content mat-focus-indicator"]')))    
            result = driver.find_elements(By.XPATH, f'//div[@class="mat-calendar-body-cell-content mat-focus-indicator"]')

            for i in result:
                if i.text == '{:d}'.format(date_1.day):
                    driver.execute_script("arguments[0].click();", i)
                    break




        if date_2.strftime("%b %Y").upper() == driver.find_element(By.CSS_SELECTOR, '.mat-calendar-period-button > span:nth-child(1)').text :
            result = driver.find_elements(By.XPATH, f'//div[@class="mat-calendar-body-cell-content mat-focus-indicator"]')
            for i in result:
                if i.text == '{:d}'.format(date_2.day):
                    driver.execute_script("arguments[0].click();", i)
                    break
        
        else:
            
            button_pass = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button.mat-focus-indicator:nth-child(4)")))
            driver.execute_script("arguments[0].click();", button_pass)
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, f'//div[@class="mat-calendar-body-cell-content mat-focus-indicator"]')))
            result = driver.find_elements(By.XPATH, f'//div[@class="mat-calendar-body-cell-content mat-focus-indicator"]')
            for i in result:
                if i.text == '{:d}'.format(date_2.day):
                    time.sleep(.50)
                    driver.execute_script("arguments[0].click();", i)
                    break

        try:
            WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(., 'Sem dados')]")))
            date_work += timedelta(days = 1)
            print("Sem dados para a datra: ", date_work)
            return False
        except TimeoutException:
            exportar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Exportar')]")))
            driver.execute_script("arguments[0].click();", exportar)
            xlsx_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.mat-menu-item')))
            driver.execute_script("arguments[0].click();", xlsx_button)
            return True

    def __download_integradas(self):

        driver = self.acess_integradas()
        
        date_work = self.date

        while date_work < date.today():
            
            path_to_save = f"./Integradas/{date_work.year}/{date_work.month}"
            if not os.path.exists(f'{path_to_save}/relatorio_{date_work.day}_{date_work.month}_{date_work.year}.xlsx'):
                print(f'{path_to_save}/relatorio_{date_work.day}_{date_work.month}_{date_work.year}.xlsx')

                self.__scroll(driver)

                if self.__calendar_manipulate(date_work = date_work, driver = driver):
                    
                    while not os.path.exists(f"{path_to_save}/relatorio_{'{:02d}'.format(date_work.day)}_{date_work.month}_{date_work.year}.xlsx"):
                        print("path ",f"{path_to_save}/relatorio_{'{:02d}'.format(date_work.day)}_{date_work.month}_{date_work.year}.xlsx" )
                        self.__move(date_work)
                time.sleep(3)
                driver.refresh()

            else:
                print("Data já baixada: ", date_work)
            date_work = date_work + timedelta(days = 1)
        driver.close()

            




In [6]:
teste = downaload_reports(driver = openBrowser().loguinSendKeys(),date = date(2023, 12, 20))

Data já baixada:  2023-12-20
./Integradas/2023/12/relatorio_21_12_2023.xlsx
path  ./Integradas/2023/12/relatorio_21_12_2023.xlsx
download/relatorio.xlsx
Aguardando arquivo
download/relatorio.xlsx
Aguardando arquivo
download/relatorio.xlsx
Aguardando arquivo
./Integradas/2023/12/relatorio_22_12_2023.xlsx
path  ./Integradas/2023/12/relatorio_22_12_2023.xlsx
download/relatorio.xlsx
Aguardando arquivo
./Integradas/2023/12/relatorio_23_12_2023.xlsx
Sem dados para a datra:  2023-12-24
./Integradas/2023/12/relatorio_24_12_2023.xlsx
Sem dados para a datra:  2023-12-25
./Integradas/2023/12/relatorio_25_12_2023.xlsx
Sem dados para a datra:  2023-12-26
./Integradas/2023/12/relatorio_26_12_2023.xlsx
path  ./Integradas/2023/12/relatorio_26_12_2023.xlsx
download/relatorio.xlsx
Aguardando arquivo
./Integradas/2023/12/relatorio_27_12_2023.xlsx
path  ./Integradas/2023/12/relatorio_27_12_2023.xlsx
download/relatorio.xlsx
Aguardando arquivo
download/relatorio.xlsx
Aguardando arquivo
download/relatorio.xl